In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import clingo
from clingo.control import Control
import time
import os
from helper import lines_counter
import json

In [2]:
canonical_dict = {"sudoku":os.path.join("projects","sudoku","canonical_sudoku.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","canonical_seeknumbers2.lp"),
                 "minotaur":os.path.join("projects","minotaur","canonical_minotaur1.lp"),
                 "creek":os.path.join("projects","creek","canonical_creek.lp"),                 
                 "yosenabe":os.path.join("projects","yosenabe","canonical_yosenabe.lp"),
                 "hop":os.path.join("projects","hop","canonical_hop.lp"),
                 "lights":os.path.join("projects","lights","canonical_lights.lp")}

instances_dict ={"sudoku":os.path.join("projects","sudoku","instances","ex01.lp"),
                 "seeknumbers":os.path.join("projects","seeknumbers","instances","ex01.lp"),
                 "minotaur":os.path.join("projects","minotaur","instances","level01.lp"),
                 "creek":os.path.join("projects","creek","instances","ex01.lp"),
                 "yosenabe":os.path.join("projects","yosenabe","instances","instance01.lp"),
                 "hop":os.path.join("projects","hop","instances","level1.lp"),
                 "lights":os.path.join("projects","lights","instances","test01.lp")}

solution_dict ={"sudoku":os.path.join("projects","sudoku","solutions","ex01.json"),
                 "seeknumbers":os.path.join("projects","seeknumbers","solutions","ex01.json"),
                 "minotaur":os.path.join("projects","minotaur","solutions","level01.json"),
                 "creek":os.path.join("projects","creek","solutions","ex01.json"),
                 "yosenabe":os.path.join("projects","yosenabe","solutions","solution01.txt"),
                 "hop":os.path.join("projects","hop","solutions","solution01.txt"),
                 "lights":os.path.join("projects","lights","solutions","solution01.txt")}

In [3]:
def asp_try_5(asp_file,instance):
    models = []  
    errors = []
    symbols = []
    messages = []

    def custom_logger(code, message):
        messages.append(message)
        
    def on_model(model):
        #print("Model:", model)
        models.append(model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            #for s in symbols_list:
            symbols.append(symbols_list)
        os.remove("output.txt")

    time_init = time.time() 
    asp_program = []
    # TODO: timeout of Unix systems. Wrapper around. Kill the process. 
    control = clingo.Control(['--warn=none','-t','5','-n','10'], logger=custom_logger) #, '--warn=none', '--opt-mode=optN' --time-limit=5, -t 5
    input_files = [asp_file, instance]
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
            
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    try:
        control.ground([("base", [])])
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols, messages
    
    time_final = time.time() 
    delta_time = time_final-time_init
    
    if delta_time > 5 and len(symbols) == 0:
        errors.append("Timeout")

    #print(f"Results: {models}, {errors}, {symbols}. Timedelta {delta_time}")
    return models, errors, symbols, messages

In [4]:
def get_symbols_from_solution(solution_path):        
    with open(solution_path,'r') as file:
        if ".json" in solution_path:
            data = json.load(file)
            results = data["Call"][0]["Witnesses"] #[0]["Value"]
            #print(result)
        else:
            lines = file.readlines()
            splitted = lines[0].split()
            results = [{"Value":splitted}]
    return results

In [5]:
def get_metrics(symbols, solutions):
    symbols_set = set(symbols)
    matchs = 0
    final_tp = set()
    final_fp = ()
    final_fn = ()
    for solution in solutions:
        solution_set = set(solution["Value"])
        union = len(symbols_set.union(solution_set))
        tp = solution_set.intersection(symbols_set)
        matchs_temp = len(tp)
        solution_len = len(solution_set)
        if matchs_temp >= matchs:            
            matchs = matchs_temp
            final_tp = tp
            final_fp = symbols_set.difference(solution_set)
            final_fn = solution_set.difference(symbols_set)
            solution_len = len(solution_set)
    accuracy = matchs / solution_len if solution_len != 0 else 0 
    precision = matchs / (len(final_tp) + len(final_fp)) if (len(final_tp) + len(final_fp)) != 0 else 0  
    recall = matchs / (len(final_tp) + len(final_fn)) if (len(final_tp) + len(final_fn)) != 0 else 0 
    f1_score = 2 * ((precision * recall) / (precision + recall)) if (precision + recall) != 0 else 0
    jaccard = matchs / union if union != 0 else 0
    #print(f"Accuracy: {accuracy:.4f}. Precision: {precision:.4f}. Recall: {recall:.4f}. F1: {f1_score:.4f}")
    return matchs, round(accuracy,2), round(precision,2), round(recall,2), round(f1_score,2), round(jaccard,2)

In [6]:
def repetition_counter(file):
    with open(file,'r') as f:
        lines = f.readlines()
        lines = [x for x in lines if x!='\n']
        #print(f"Lines {lines}")
        list_unique_lines = list(set(lines))
        unique_lines = len(list_unique_lines)
        #print(f"List Unique Lines: {list_unique_lines}")
        return len(lines) - unique_lines

In [7]:
def get_best_metrics(all_symbols,solutions):
    matchs = acc = prec = rec = f1 = jac = 0
    for symbols in all_symbols:
        matchs_t, acc_t, prec_t, rec_t, f1_t, jac_t = get_metrics(symbols, solutions)
        if matchs_t > matchs:
            matchs = matchs_t
            acc = acc_t
            prec = prec_t
            rec = rec_t
            f1 = f1_t
            jac = jac_t
    return matchs, acc, prec, rec, f1, jac

In [8]:
def evaluate(file, instance, solution_path, canonical_file):
    models, errors, symbols, messages = asp_try_5(file,instance)
    n_errors = len(errors)
    errors = "".join(errors)
    n_models = len(models)    
    canonical_lines = lines_counter(canonical_file)
    encoding_lines = lines_counter(file)
    repeated_lines = repetition_counter(file)
    #print("Length of Canonical:" + str(canonical_lines) + ". Length Encoding: "+ str(encoding_lines))    
    diff_lines = abs(canonical_lines - encoding_lines)
    solutions = get_symbols_from_solution(solution_path)
    #print("Symbols: " + str(symbols))
    #print("Solution: " + str(solution))
    matchs, accuracy, precision, recall, f1_score, jaccard = get_best_metrics(symbols, solutions) 
    at_least_a_model = 1 if n_models > 0 else 0
    kpi = 25 + at_least_a_model*25 + 10*(accuracy + precision + recall + f1_score + jaccard) - 2*(diff_lines + repeated_lines + n_errors)
    result_dict = {"kpi":kpi, "diff_lines":diff_lines, "repeated":repeated_lines, "n_errors":n_errors, "n_models":n_models, "errors":errors, "symbols":symbols, "messages":messages, "matchs":matchs,
                    "accuracy":accuracy, "precision":precision, "recall":recall, "f1_score":f1_score, "jaccard":jaccard}
    #print(f"Precision: {precision}")
    print_out_dict = {x: result_dict[x] for x in result_dict if x not in ["symbols"]}
    print(print_out_dict)
    return result_dict

In [15]:
class TextEditor:
    def __init__(self, root):
        self.root = root
        self.root.title("Text File Editor")
        self.root.geometry("1500x600")

        # Main frame for splitting the window
        self.main_frame = tk.PanedWindow(root, orient=tk.HORIZONTAL)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Create a Text Widget for editing
        self.text_area = scrolledtext.ScrolledText(self.main_frame, wrap=tk.WORD)
        self.main_frame.add(self.text_area)

        # Create a Text Widget for display only
        self.display_area = scrolledtext.ScrolledText(self.main_frame, wrap=tk.WORD)
        self.display_area.config(state=tk.DISABLED)  # Make it read-only
        self.main_frame.add(self.display_area)

        # Add a Menu Bar for file operations
        menu_bar = tk.Menu(root)
        root.config(menu=menu_bar)

        file_menu = tk.Menu(menu_bar, tearoff=0)
        file_menu.add_command(label="Open", accelerator="Ctrl+O", command=self.open_file)
        #file_menu.add_command(label="Open View-Only", accelerator="Ctrl+Alt+O", command=self.open_view_only)
        file_menu.add_command(label="Test", accelerator="Ctrl+T", command=self.test_clingo)
        file_menu.add_command(label="Save As Modified", accelerator="Ctrl+S", command=self.save_file)

        menu_bar.add_cascade(label="File", menu=file_menu)

        # Bind shortcut keys
        self.root.bind("<Control-o>", lambda event: self.open_file())
        #self.root.bind("<Control-Alt-o>", lambda event: self.open_view_only())
        self.root.bind("<Control-t>", lambda event: self.test_clingo())
        self.root.bind("<Control-s>", lambda event: self.save_file())
    
    def open_file(self):
        """ Open a file for editing. """
        filepath = filedialog.askopenfilename(filetypes=[("ASP files", "*.lp"), ("Text files", "*.txt"), ("All files", "*.*")])
        if not filepath:
            return
        self.filepath = filepath
        self.filename = os.path.basename(filepath)
        self.root.title(f"Text File Editor - {self.filename}")
        with open(self.filepath, 'r') as file:
            text = file.read()
            self.text_area.delete('1.0', tk.END)
            self.text_area.insert(tk.END, text)
        # Display the other story in the other panel
        story = self.filename.split("_")[0]
        canonical_path = canonical_dict[story]
        with open(canonical_path, 'r') as file:
            text = file.read()
            self.display_area.config(state=tk.NORMAL)
            self.display_area.delete('1.0', tk.END)
            self.display_area.insert(tk.END, text)
            self.display_area.config(state=tk.DISABLED)      

    def save_file(self):
        """ Save the current file as modified. """
        if hasattr(self, 'filepath') and self.filepath:
            new_filepath = f"{self.filepath.rsplit('.', 1)[0]}_modified.lp"
            with open(new_filepath, 'w') as file:
                text = self.text_area.get('1.0', tk.END)
                file.write(text)
            messagebox.showinfo("Save", "File saved successfully!")
        else:
            messagebox.showerror("Error", "There was an error saving the file.")

    def test_clingo(self):
        """ Test if the current programm can solve the problem"""
        text = self.text_area.get('1.0', tk.END)
        test_file = "test_file.lp"
        with open(test_file,"w") as file:
            file.write(text)
        story = self.filename.split("_")[0]
        results_dict = evaluate(test_file,instances_dict[story],solution_dict[story],canonical_dict[story])
        os.remove(test_file)
        messagebox.showinfo("Dict", str(results_dict))
        line_count = len(text.strip().split('\n'))
        if results_dict["jaccard"] == 1:
            messagebox.showinfo("Result", "Total success!")
        else:
            messagebox.showerror("Result", "It fails")

In [16]:
def main():
    root = tk.Tk()
    app = TextEditor(root)
    root.mainloop()

if __name__ == "__main__":
    main()